In [1]:
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from statistics import mean, mode, median
from matplotlib import cm, colors
from distfit import distfit
from inflection import humanize, underscore
import json
from math import sqrt
import statistics


%matplotlib inline

In [2]:
def aparar_arr(arr, porcentagem):
    index_inf = round(len(arr) * porcentagem) - 1
    index_sup = round(len(arr) * (1-porcentagem)) - 1
    return arr[index_inf:index_sup]

In [37]:
def plotHist(dataFrame, metric, path, title, ext):
    data = dataFrame[metric].to_numpy()
    plt.hist(data, bins=np.unique(data), log=True)
    plt.title(title)
    plt.xlabel("Metric value")
    plt.ylabel("Frequency")
    if metric != 'lcom*':
        plt.savefig(path+'/hist_'+metric+ ext+'.png')
    else:
        plt.savefig(path+'/hist_'+'lcomStar'+ ext+'.png')
    plt.close()

In [27]:
def printHist(dataFrame, metric, title):
    data = dataFrame[metric].to_numpy()
    plt.hist(data, bins=np.unique(data), log=True)
    plt.title(title)
    plt.xlabel("Metric value")
    plt.ylabel("Frequency")
    plt.show()
    plt.close()

### Tabela com todos os dados

In [5]:
import pandas as pd
import os
project_path = 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/metrics/'
dtframes = []
for file in os.listdir(project_path):
    if file.endswith("class.csv"):
        dfTemp = pd.read_csv(os.path.join(project_path, file))
        dtframes.append(dfTemp)


df = pd.concat(dtframes, axis=0, ignore_index=True)

In [6]:
import functools
from math import sqrt

def calculate_metrics_on_df(dataframe_input, porcentagem):
    dfWithDataOnly = dataframe_input.drop(columns=['file', 'class', 'type', 'assignmen'])
    mean = dfWithDataOnly.mean()
    median = dfWithDataOnly.median()
    mode = dfWithDataOnly.mode().T
    mode['metric'] = mode.index
    std = dfWithDataOnly.std()
    percentile70 = dfWithDataOnly.quantile(0.7)
    percentile90 = dfWithDataOnly.quantile(0.9)

    x = porcentagem
    k = sqrt(-1/(x - 1))
    limite_inf = mean - k*std
    limite_sup = mean + k*std

    meanDf = pd.DataFrame({'metric': mean.index, 'avg': mean.values})
    medianDf = pd.DataFrame({'metric': mean.index,'median': median.values})
    stdDf = pd.DataFrame({'metric': std.index, 'std': std.values})
    modeDf = mode.set_axis(['mode', 'metric'], axis=1)
    percentile70Df = pd.DataFrame({'metric': percentile70.index,'percentile70': percentile70.values})
    percentile90Df = pd.DataFrame({'metric': percentile90.index,'percentile90': percentile90.values})

    limite_infDf = pd.DataFrame({'metric': limite_inf.index, 'limite_inf': limite_inf.values})
    limite_supDf = pd.DataFrame({'metric': limite_sup.index, 'limite_sup': limite_sup.values})
    limites_merged = df_merged = pd.merge(limite_infDf, limite_supDf, on='metric')
    limites_merged['chebyshev'] = list(zip(df_merged['limite_inf'], df_merged['limite_sup']))
    chebyshev = limites_merged.drop(columns=['limite_inf', 'limite_sup'])

    metrics = [meanDf, stdDf, medianDf, modeDf, percentile70Df, percentile90Df, chebyshev]
    df_final = functools.reduce(lambda left, right: pd.merge(left,right,on='metric'), metrics)
    return df_final.sort_values(by='metric')

In [7]:
df_data_ia = calculate_metrics_on_df(df, 0.6)

### Tabela com os dados > 1

In [8]:
df_data_ia_without_zeros = calculate_metrics_on_df(df.replace(0, np.NaN), 0.6)
df_data_ia_without_zeros

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
19,abstractMethodsQty,2.427768,3.603013,1.000000,1.0,2.0,5.0,"(-3.269095458830206, 8.12463227726753)"
43,anonymousClassesQty,2.323222,3.748823,1.000000,1.0,2.0,5.0,"(-3.604186723754744, 8.250631278702532)"
39,assignmentsQty,19.288867,91.631096,6.000000,1.0,14.0,40.0,"(-125.59261641843194, 164.17035082018526)"
0,cbo,7.294427,7.948355,5.000000,2.0,8.0,16.0,"(-5.27302490849757, 19.86187918333315)"
1,cboModified,9.953587,12.580825,7.000000,2.0,11.0,20.0,"(-9.93844378376836, 29.84561757911216)"
34,comparisonsQty,6.138124,14.579225,3.000000,1.0,5.0,13.0,"(-16.913653712637295, 29.189902438427307)"
27,defaultFieldsQty,2.479971,4.031514,1.000000,1.0,2.0,5.0,"(-3.894412453722435, 8.854354608458516)"
17,defaultMethodsQty,4.003446,10.126523,2.000000,1.0,3.0,8.0,"(-12.007993622114888, 20.01488524164548)"
5,dit,1.654189,2.480641,1.000000,1.0,2.0,3.0,"(-2.268048254264796, 5.576426377911534)"
2,fanin,4.913965,11.107714,2.000000,1.0,4.0,10.0,"(-12.648872615365883, 22.47680214120247)"


### Tabela com dados aparados 5%

In [9]:
project_path = '../../Projetos/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    new_data = aparar_arr(data, 0.05)
    values = [metric]
    values.extend(statistic_metric(new_data,0.6))
    arr.append(values)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../Projetos/'

In [10]:
df_data5 = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data5

NameError: name 'arr' is not defined

### Tabela com dados aparados 10%

In [ ]:
project_path = '../../Projetos/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    new_data = aparar_arr(data, 0.1)
    values = [metric]
    values.extend(statistic_metric(new_data,0.6))
    arr.append(values)

wmc: 100%|██████████| 44/44 [00:22<00:00,  1.99it/s]


In [ ]:
df_data10 = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data10

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
1,anonymousClassesQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
2,assignmentsQty,3.707516,4.862352,2.000000,0.0,4.000000,11.000000,"(-3.980536617932502, 11.395569060185291)"
3,cbo,6.469338,5.632936,5.000000,0.0,9.000000,15.000000,"(-2.437116423327888, 15.375792779414054)"
4,comparisonsQty,0.352603,0.785381,0.000000,0.0,0.000000,1.000000,"(-0.8891932701438121, 1.5943996028715566)"
5,defaultFieldsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
6,defaultMethodsQty,0.126348,0.332241,0.000000,0.0,0.000000,1.000000,"(-0.39897161941958137, 0.6516682194714889)"
7,dit,1.393418,0.660188,1.000000,1.0,1.000000,2.000000,"(0.34956985950404085, 2.437266372003875)"
8,fanin,1.039792,1.565162,0.000000,0.0,1.000000,3.000000,"(-1.4349462151456462, 3.514530954309934)"
9,fanout,4.422419,4.028875,3.000000,0.0,6.000000,11.000000,"(-1.9477920112073024, 10.792629799943352)"


In [11]:
df_data_ia.to_csv('./Statistic_Data_Complete.csv', sep=';')
df_data_ia_without_zeros.to_csv('./Statistic_Data_Without_0.csv', sep=';')

### Histogramas de NOC

In [26]:
hist_path = 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/statisticTable/histogram'
plotHist(df.replace(0, np.NaN), 'noc', hist_path,'Histograma NOC dataset completo', '')

In [39]:
df_data_ia_without_zeros

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
19,abstractMethodsQty,2.427768,3.603013,1.000000,1.0,2.0,5.0,"(-3.269095458830206, 8.12463227726753)"
43,anonymousClassesQty,2.323222,3.748823,1.000000,1.0,2.0,5.0,"(-3.604186723754744, 8.250631278702532)"
39,assignmentsQty,19.288867,91.631096,6.000000,1.0,14.0,40.0,"(-125.59261641843194, 164.17035082018526)"
0,cbo,7.294427,7.948355,5.000000,2.0,8.0,16.0,"(-5.27302490849757, 19.86187918333315)"
1,cboModified,9.953587,12.580825,7.000000,2.0,11.0,20.0,"(-9.93844378376836, 29.84561757911216)"
34,comparisonsQty,6.138124,14.579225,3.000000,1.0,5.0,13.0,"(-16.913653712637295, 29.189902438427307)"
27,defaultFieldsQty,2.479971,4.031514,1.000000,1.0,2.0,5.0,"(-3.894412453722435, 8.854354608458516)"
17,defaultMethodsQty,4.003446,10.126523,2.000000,1.0,3.0,8.0,"(-12.007993622114888, 20.01488524164548)"
5,dit,1.654189,2.480641,1.000000,1.0,2.0,3.0,"(-2.268048254264796, 5.576426377911534)"
2,fanin,4.913965,11.107714,2.000000,1.0,4.0,10.0,"(-12.648872615365883, 22.47680214120247)"


In [38]:
metrics = df_data_ia_without_zeros['metric']
for metric in metrics:
    plotHist(df, metric, hist_path, 'Histograma'+metric+' dataset', '')

### Tabela com dados
Sem o valor 0, caso a métrica seja discreta e o percentil 70 tenha sido 0 também.

In [ ]:
metrics_without_zero = ['abstractMethodsQty', 'anonymousClassesQty', 'comparisonsQty','defaultFieldsQty', 'defaultMethodsQty','finalMethodsQty', 'innerClassesQty', 'lambdasQty', 'logStatementsQty', 'loopQty', 'mathOperationsQty', 'noc', 'parenthesizedExpsQty', 'privateMethodsQty', 'protectedFieldsQty', 'protectedMethodsQty', 'publicFieldsQty', 'staticFieldsQty', 'staticMethodsQty', 'synchronizedFieldsQty', 'synchronizedMethodsQty', 'tryCatchQty']

In [ ]:
df.to_csv('./Statistic_Data_New_Approach.csv', sep=';', decimal=",")

1 - gerar gráficos para todas as métricas utilizadas pelo Diego
2 - Detalhar metodologia
3 - Fazer as análises e escrever resultados